In [ ]:
import requests
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt

In [ ]:
start_date = '2023-06-01'
end_date = '2023-06-30'
base = 'BTC'
symbols = 'RUB'
format = 'CSV'

Создадим переменную response и запишем в нее ответ сервера. Сохраним результат в формате *.csv.

In [ ]:
response = requests.get('https://api.exchangerate.host/timeseries?',
                            params={'base': base,
                                    'start_date': start_date,
                                    'end_date': end_date,
                                    'symbols': symbols,
                                    'format': format
                            })

with open('./csv_files/exchange_june2023.csv', 'wb') as f:
    f.write(response.content)

Замена разделителя десятичных знаков с "," на ".", для корректной загрузки в базу данных.

In [ ]:
df = pd.read_csv('./csv_files/exchange_june2023.csv', decimal=',', index_col=False)
df = pd.DataFrame(df)
df

,date,code,rate,base,start_date,end_date
0,2023-06-01,RUB,2.180140e+06,BTC,2023-06-01,2023-06-30
1,2023-06-02,RUB,2.219636e+06,BTC,2023-06-01,2023-06-30
2,2023-06-03,RUB,2.164137e+06,BTC,2023-06-01,2023-06-30
3,2023-06-04,RUB,2.216854e+06,BTC,2023-06-01,2023-06-30
4,2023-06-05,RUB,2.060827e+06,BTC,2023-06-01,2023-06-30
5,2023-06-06,RUB,2.231097e+06,BTC,2023-06-01,2023-06-30
6,2023-06-07,RUB,2.128601e+06,BTC,2023-06-01,2023-06-30
7,2023-06-08,RUB,2.157652e+06,BTC,2023-06-01,2023-06-30
8,2023-06-09,RUB,2.168827e+06,BTC,2023-06-01,2023-06-30
9,2023-06-10,RUB,2.117741e+06,BTC,2023-06-01,2023-06-30


Сохраним таблицу в файл *.csv.

In [ ]:
df.to_csv('./csv_files/exchange_june2023.csv', index=False)

Подключение к базе Postgres, развернутой в docker-compose:

In [ ]:
conn = psycopg2.connect(database='exchange',
                        user='postgres',
                        password='password',
                        host='localhost',
                        port=5434)
cur = conn.cursor()

In [ ]:
cur.execute("""
CREATE TABLE exchange_june2023(
    date DATE PRIMARY KEY,
    exchange_id VARCHAR,
    exchange_rate DECIMAL,
    base_exchange VARCHAR,
    start_date DATE,
    end_date DATE
)
""")

Заполнение таблицы данными

In [ ]:
cur.execute("""COPY exchange_june2023
FROM '/docker-entrypoint-initdb.d/csv_files/exchange_june2023.csv'
DELIMITER ','
CSV HEADER;""")
conn.commit()

In [ ]:
cur.execute("""select * from exchange_june2023;""")
print(cur.fetchone())
conn.commit()

(datetime.date(2023, 6, 1), 'RUB', Decimal('2180140.15'), 'BTC', datetime.date(2023, 6, 1), datetime.date(2023, 6, 30))


Создание таблицы c результатами по курсу биткоина (BTC):
   - день, в который значение курса было максимальным (date_max_rate);
   - максимальное значение курса (max_val_rate);
   - день, в который значение курса было минимальным, (date_min_rate);
   - минимальное значение курса (min_val_rate);
   - среднее значение курса за весь месяц, (avg_rate);
   - значение курса на последний день месяца. (last_date_rate).

In [ ]:
cur.execute("""CREATE TABLE result_query AS (SELECT * FROM (SELECT "date" date_max_rate, MAX(exchange_rate) max_val_rate
				FROM exchange_june2023
				WHERE exchange_rate = (SELECT MAX(exchange_rate) FROM exchange_june2023)
				GROUP BY "date") max_rate
CROSS JOIN
	(SELECT "date" date_min_rate, MIN(exchange_rate) min_val_rate
	FROM exchange_june2023
	WHERE exchange_rate = (SELECT MIN(exchange_rate) FROM exchange_june2023)
	GROUP BY "date") min_rate
CROSS JOIN
	(SELECT AVG(exchange_rate) avg_rate FROM exchange_june2023) avg_rate
CROSS JOIN
	(SELECT exchange_rate last_date_rate
						FROM exchange_june2023
						WHERE "date" = '2023-06-30') last_date_rate)
;""")
conn.commit()

Отображение таблицы "result_query"

In [ ]:
cur.execute("""select * from result_query;""")
print(cur.fetchall())
conn.commit()
conn.close ()

[(datetime.date(2023, 6, 30), Decimal('2693520.583333'), datetime.date(2023, 6, 5), Decimal('2060826.642857'), Decimal('2312089.860954800000'), Decimal('2693520.583333'))]
